In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import YieldSimulator
from elfpy.utils.data import format_trades

In [ ]:
pd.set_option("float_format",'{:,.3f}'.format)

In [ ]:
start_time = time.time()

config_file = "../../config/simulation_config.toml"
simulator = YieldSimulator(config_file)
simulator_rng = np.random.default_rng(simulator.config.simulator.random_seed)
simulator.reset_rng(simulator_rng)
simulator.set_random_variables()

num_runs = 200 # lots of runs to have high statistical power
for run_number in range(num_runs): # run a simulation per pricing model
    override_dict = {
        'pricing_model_name': 'YieldSpacev2',
        #'vault_apy': [0.1,]*config['num_trading_days'],
        #'init_vault_age': 1,
        #'init_pool_apy' : 0.05,
        #'target_liquidity': 3e6,
        #'fee_percent' : 0.1,
    }
    simulator.run_simulation(override_dict)

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
[trades, trades_agg] = format_trades(simulator.analysis_dict)

In [ ]:
print(f'Simulator contains {trades.shape[1]} variables tracked over {trades.shape[0]} simulations.')
print(f'Trades dataframe variables:\n'+'\t'.join(list(trades.columns)))
print(f'\nTrades dataframe:')
display(pd.concat([trades.head(4), trades.tail(2)]).T)
print(f'Rows at maturity:')
display(trades.loc[(trades.time_diff>0) | (trades.time_diff_shift>0) | (trades.index == trades.index.max()), :])

In [ ]:
# create runs dataframe from the last trade in each run (using time_diff instead would pick the 1st)
changing_columns = [ # These columns should change with run aggregation
    'run_number',
    'model_name',
    'num_orders',
    'pool_apy',
]
const_columns = [ # These should not change with run aggregation
    'init_time_stretch',
    'target_liquidity',
    'target_daily_volume',
    'fee_percent',
    'init_vault_age',
    'vault_apy',
    'pool_duration',
    'num_trading_days'
] 
interesting_columns = changing_columns + const_columns
runs = trades.loc[trades.time_diff_shift > 0, :].loc[:, interesting_columns].copy()
# variables that change per trade represent the last value in the run, rename the useful ones
runs.rename(columns={'current_apy':'ending_apy', 'num_orders':'total_orders'}, inplace=True)
# there is one row per "run", set that as a named column
runs.set_index('run_number', inplace=True)
display(runs.T)

In [ ]:
print('Standard deviations of constant columns (should all be zero):')
display(trades.groupby('run_number')[[x for x in const_columns]].std().T) # check if these are really constant

In [ ]:
# target columns for taking averages & sums
mean_columns = [
    'time_until_end',
    'init_share_price',
    'base_asset_price',
    'spot_price',
    'out_without_fee_slippage',
    'base_asset_reserves',
    'out_without_fee',
    'fee',
    'trade_amount',
    'out_with_fee',
    'day',
    'fee_in_bps',
    'token_asset_reserves',
    'total_supply'
]
sum_columns = [
    'fee',
    'out_with_fee',
    'out_without_fee',
    'out_without_fee_slippage',
    'trade_amount'
]
trades_mean = trades.groupby(['run_number'])[mean_columns].mean()
trades_mean.columns = ['mean_'+col for col in trades_mean.columns]
trades_sum = trades.groupby(['run_number'])[sum_columns].sum()
trades_sum.columns = ['sum_'+col for col in trades_sum.columns]
display(pd.concat([runs, trades_mean, trades_sum], axis=1).T)

In [ ]:
display(trades.loc[:, ['day', 'num_orders', 'trade_amount', 'spot_price', 'price_total_return', 'price_total_return_scaled_to_share_price']])

In [ ]:
display(trades.groupby(['run_number', 'token_in']).agg({
    'trade_volume_usd': ['sum'],
    'num_orders': ['count'],
    'fee_in_bps': ['mean', 'std', 'min', 'max', 'sum']
    }).rename(columns={'num_orders_sum': 'trade_number_sum'})
)

In [ ]:
# investigate distribution of direction of trades across multiple runs
aggregated_by_one_direction=trades.loc[trades.token_in=='base',:].groupby(['run_number', 'day']).agg({'run_trade_number':'count'}).reset_index()\
    .rename(columns={'run_trade_number':'number_of_days_with_this_many_base_trades'})
# display(aggregated_by_one_direction)
aggregated_by_one_direction_historgram=pd.crosstab(index=aggregated_by_one_direction.run_number,\
    columns=aggregated_by_one_direction.number_of_days_with_this_many_base_trades)
display(aggregated_by_one_direction_historgram.sort_values(by=10, ascending=False))
plt.bar(range(1,12), aggregated_by_one_direction_historgram.sum())
plt.title('Number of days with X base trades')
plt.show()

In [ ]:
xtab = pd.crosstab(index=trades_agg.day, columns=trades_agg.day, values=trades_agg.spot_price_mean, aggfunc='mean')
# xtab_count = pd.crosstab(index=trades.day, columns=trades.token_in)
xtab_count = pd.crosstab(index=trades.day, columns=trades.day, values=trades.token_in.values=='base', aggfunc='sum')
xtab_count.quantile([.01, .05, .90, .95])
xtab_big = xtab.mean(axis=1)
xtab_std = xtab.std(axis=1)
xtab_ptiles = xtab.quantile([0.05, 0.1, 0.90, 0.95], axis=1)
xtab_all = pd.concat([xtab_big, xtab_std], axis=1)
xtab_all.rename(columns={0: 'mean', 1: 'std'}, inplace=True)
# xtab_count['total'] = xtab_count.loc[:,'base'] + xtab_count.loc[:,'fyt']
# xtab_count['base_percent_of_total'] = xtab_count.loc[:,'base'] / xtab_count.loc[:,'total']
# display(xtab_count.sort_values(by='base_percent_of_total', ascending=False))
display(xtab_count.quantile([.01, .05, .90, .95],axis=1).T)

In [ ]:
idx = np.repeat([-2,-1,1,2], [xtab_all.shape[0]]).reshape(4, xtab_all.shape[0]).T
# display(xtab_all.loc[:,'std'].multiply([-2,1,1,2], axis=0))
idx.T.dot(xtab_all.loc[:,'std'])

In [ ]:
numPlots = 1

figsize = (24, 18)
#fig, ax = plt.subplots(ncols=1, nrows=numPlots, gridspec_kw = {'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
fig, ax = plt.figure(figsize=figsize), plt.gca()
fig.patch.set_facecolor('white')   # set fig background color to white
if numPlots==1: ax = [ax,ax]

# colors = list(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# model_colors = {model:colors[i] for i, model in enumerate(trades_agg.model_name.unique())}

# spot_price
current_plot = 0
plt.plot(xtab, color = 'grey', alpha = 0.1)
plt.plot(xtab_big, color='black', linewidth=3)
# plt.plot(xtab_ptiles.T, color='black', linewidth=2)
#plt.plot(xtab_all.mean + xtab_all.loc[:,'std'].mul(idx, axis=0), color='black', linewidth=2)
# xtab_all.plot(ax=ax[current_plot], linewidth=3, y=['mean'], color='black', yerr='std', capsize=5)
ax[current_plot].set_xlabel('')
ax[current_plot].set_ylabel('Principal Token Price', fontsize=18)
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
title = (
    f'Principal Token Price\n'
    +f'Initial pool APY: {trades["pool_apy"][0]:.2f}%, Time Stretch: {trades["init_time_stretch"][0]:.2f}, Maturity: {trades["days_until_end"][0]} days\n'
    +f'Target Liquidity: {trades["target_liquidity"][0]}, Target Daily Volume: {trades["target_daily_volume"][0]}, Percent Fees: {trades["fee_percent"][0]:.2f}%'
)

ax[current_plot].set_title(title, fontsize=20)
ax[current_plot].legend(fontsize=18)

# format axes
ax[current_plot].tick_params(axis='both', labelsize=18)
ax[current_plot].grid(visible=True, linestyle='--', linewidth='1', color='grey', which='both', axis='y')
ax[current_plot].xaxis.set_ticklabels([])
ax[current_plot].legend(fontsize=18)
ax[current_plot].set_xlabel('Day', fontsize=18)

plt.show()